In [77]:
from sklearn import svm
from lib import f1_score
import pickle
import numpy as np
import os
from sklearn.utils import multiclass as mc
from sklearn.multiclass import OneVsRestClassifier

from sklearn.preprocessing import MultiLabelBinarizer


data_dir = "../data/"
pickle_file = 'tf_data_2.pickle'


with open(data_dir + pickle_file, 'rb') as f:
    save = pickle.load(f)
    
    print save
    train_dataset = np.matrix(save['train_data'], dtype='float32')
    train_labels = np.matrix(save['train_labels'], dtype='float32')
    valid_dataset = np.matrix(save['validate_data'], dtype='float32')
    valid_labels = np.matrix(save['validate_labels'], dtype='float32')
    test_dataset = np.matrix(save['test_data'], dtype='float32')
    test_bids = list(save['test_business_ids'])
    #test_bids = np.ravel(test_bids)
    del save  # hint to help gc free up memory
    print 'Training set', train_dataset.shape, train_labels.shape
    print 'Validation set', valid_dataset.shape, valid_labels.shape
    print 'Test set', test_dataset.shape, len(test_bids)

def matrix_2_labels(x, label_map=None):
    x_len, x_width = x.shape
    out = [None] * x_len
    for i in range(x_len):
        out[i] = list()
        for label in range(x_width):
            #print x[i][0][0]
            if x.item((i, label)) > 0:
                if label_map == None:
                    out[i].append(label)
                else:
                    out[i].append(label_map[label])

    return out

OneVsRestClassifier

train_labels = MultiLabelBinarizer().fit_transform(matrix_2_labels(train_labels))
#train_labels = matrix_2_labels(train_labels)
print mc.type_of_target(train_labels)
train_labels = np.asarray(train_labels)
print train_labels.shape

#clf = svm.SVC(decision_function_shape='ovr', kernel='linear')

clf = OneVsRestClassifier(svm.LinearSVC(random_state=0)).fit(train_dataset, train_labels)
predictions = clf.predict(valid_dataset)
print f1_score.mean_f1_score(predictions, valid_labels)


{'train_labels': matrix([[ 1.,  1.,  0., ...,  0.,  0.,  0.],
        [ 1.,  1.,  1., ...,  0.,  0.,  0.],
        [ 1.,  1.,  1., ...,  0.,  0.,  0.],
        ..., 
        [ 1.,  1.,  1., ...,  0.,  0.,  0.],
        [ 1.,  1.,  1., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float16), 'validate_data': matrix([[ 0.65136719,  1.26953125,  0.18884277, ...,  0.60693359,
          0.37304688,  0.41601562],
        [ 1.01171875,  0.68798828,  0.03851318, ...,  0.33544922,
          0.10107422,  0.70654297],
        [ 0.70214844,  1.0625    ,  0.20544434, ...,  0.22509766,
          0.30664062,  0.29882812],
        ..., 
        [ 0.65869141,  0.84570312,  0.08081055, ...,  0.87060547,
          0.12854004,  0.46704102],
        [ 0.69384766,  1.14453125,  0.19750977, ...,  0.24169922,
          0.28979492,  0.20788574],
        [ 1.07617188,  0.18762207,  0.12188721, ...,  1.47167969,
          0.00659561,  0.10015869]], dtype=float16), 'test_business_ids': [

In [78]:
predictions = clf.predict(train_dataset)
print f1_score.mean_f1_score(predictions, train_labels)

0.996516965673


In [79]:
predictions = clf.predict(valid_dataset)
print f1_score.mean_f1_score(predictions, valid_labels)

0.86466974692


In [80]:
predictions = clf.predict(test_dataset)

In [71]:
print predictions
test_results = predictions

[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]]


In [72]:
#print test_bids


test_results = np.array(np.round(test_results), dtype=bool)
output = "business_id,labels\n"

for i in range(len(test_bids)):
    print i
    tags = np.argwhere(test_results[i])
    tags_str = ' '.join('%d'%F for F in tags[:] )
    output += test_bids[i]+ "," + tags_str + "\n"
    
print output
f = open('test_submission_svm_avg2', 'w')
f.write(output)
f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [76]:
predictions[1]

array([1, 1, 1, 1, 1, 1, 0, 0, 0])